In [1]:
# Dependancies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
#! pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\chris\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Step 1 - Scraping

## NASA Mars News

In [ ]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [ ]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
element = news_soup.select_one('div.list_text')

In [ ]:
news_title = element.find('div',class_='content_title').get_text()
news_title

In [ ]:
news_p = element.find('div',class_='article_teaser_body').get_text()
news_p

## JPL Mars Space Images - Featured Image

In [ ]:
img_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(img_url)

In [ ]:
img_button = browser.find_by_css('button.btn.btn-outline-light')
# browser.find_by_text("Full Image")
# browser.find_by_css("")
img_button.click()

In [ ]:
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
full_img_url = img_soup.find('img', class_='fancybox-image').get('src')
full_img_url

In [ ]:
full_img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{full_img_url}'
full_img_url

## Mars Facts

In [ ]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [ ]:
mars_facts = pd.read_html(facts_url)[0]
mars_facts = mars_facts.rename(columns={0:'Measure',1:'Value'})
mars_facts.set_index('Measure')

In [ ]:
mars_facts = mars_facts.to_html()
mars_facts

## Mars Hemispheres

In [ ]:
base_url = 'https://astrogeology.usgs.gov'
mars_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_url)

html = browser.html
mars_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# List all hemisphere items
itemLink_ls = mars_soup.find_all('div',class_='description')

hemisphere_image_urls = []

itemLink_url = ''
title = ''
full_img_url = ''

# Loop through hemisphere items
for item in itemLink_ls:
    if (item.a):
        itemLink_url = item.a.get('href')
        itemLink_url = f'{base_url}{itemLink_url}'
        if (item.a.h3):
            title = item.a.h3.get_text()
    
    # Get full image url
    browser.visit(itemLink_url)
    html = browser.html
    hem_soup = BeautifulSoup(html, 'html.parser')
    
    x = hem_soup.find('div', class_='downloads')
    if (x.ul):
        if (x.li):
            if (x.a):
                full_img_url = x.ul.li.a.get('href')
    
    # Apend to list
    hemisphere_image_urls.append({'Title':title, 'Image_URL':full_img_url})
    
    # Reset variables
    itemLink_url = ''
    title = ''
    full_img_url = ''   
    
hemisphere_image_urls

# Step 2 - MongoDB and Flask Application

# Step 3 - Submission